In [2]:
import ijson
import numpy as np
import cv2
import urllib.request
from PIL import Image
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import torch
from pinecone import Pinecone
import json

In [15]:
import time
import random

In [3]:
torch.__version__

'2.3.0'

In [4]:
torch.cuda.get_device_name(torch.cuda.current_device())

'NVIDIA GeForce RTX 4090'

In [4]:
pinecone_api_key = "92fd371f-5b1e-415b-bcb4-9a9922514120"
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index("clip-embed") # don't mess up the name here

In [5]:
pinecone_index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [6]:
model = SentenceTransformer('clip-ViT-B-32')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

SentenceTransformer(
  (0): CLIPModel()
)

In [7]:
json_file_test = "output/test_images.json"

In [12]:
with open(json_file_test, 'r') as f:
    data = json.load(f)



In [14]:
for key,value in data.items():
    print(key)
    print(value['image_link'], value['title'])
    break

0000099813
https://m.media-amazon.com/images/I/3196g3bMM5L._AC_.jpg Military US Issue JP 8 Gore TEX Fuel Handler Coverall L/R (Large)


In [16]:
def fetch_image(url):
    """Attempt to fetch an image from a URL with retries for reliability."""
    max_attempts = 5
    for attempt in range(max_attempts):
        try:
            response = urllib.request.urlopen(url)
            image = np.asarray(bytearray(response.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            return Image.fromarray(image)
        except Exception as e:
            print(f"Attempt {attempt+1}: Failed to load {url}: {str(e)}")
            time.sleep(2 ** attempt + random.random())  # Exponential backoff + jitter
    return None

def process_images_in_batches(json_file, batch_size=10):
    batch = []
    with open(json_file, 'rb') as file:
        items = ijson.items(file, 'item')
        for item in items:
            image_link = item['image_link']
            key = item['id']
            title = item['title']
            image = fetch_image(image_link)
            if image:
                embeddings = model.encode([image], convert_to_tensor=True)
                batch.append({"id": key, "values": embeddings.cpu().numpy().tolist(), "metadata": {"title": title}})
            
            if len(batch) >= batch_size:
                pinecone_index.upsert(vectors=batch)
                print(f"Upserted batch of size {len(batch)}")
                batch = []
        
        if batch:  # Upsert any remaining images
            pinecone_index.upsert(vectors=batch)
            print(f"Upserted final batch of size {len(batch)}")


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_large_json_file.json'

In [18]:
process_images_in_batches('output/test_images.json', batch_size=128)

In [25]:
def fetch_images(urls):
    images = []
    for url in tqdm(urls):
        try:
            resp = urllib.request.urlopen(url)
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(Image.fromarray(image))
        except Exception as e:
            print(f"Failed to load {url}: {str(e)}")
    return images

def upsert_to_pinecone(pinecone_index, batch):
    if batch:
        pinecone_index.upsert(vectors=batch)

def process_images_in_batches(json_file, batch_size=10):
    urls = []
    keys = []
    titles = []
    count = 0

    with open(json_file, 'r') as file:
        for item in ijson.items(file, 'item'):
            urls.append(item['image_link'])
            keys.append(item['id'])
            titles.append(item['title'])
            count += 1

            if count >= batch_size:
                images = fetch_images(urls)
                embeddings = model.encode(images, batch_size=len(images), convert_to_tensor=True, show_progress_bar=True)
                batch = [
                    {"id": key, 
                     "values": embedding.cpu().numpy().tolist(), "metadata": {"title": title}}
                    for key, embedding, title in zip(keys, embeddings, titles)
                ]
                upsert_to_pinecone(pinecone_index, batch)
                print(f"Upserted batch of size {len(images)}")
                
                urls = []
                keys = []
                titles = []
                count = 0

        if urls:
            images = fetch_images(urls)
            embeddings = model.encode(images, batch_size=len(images), convert_to_tensor=True, show_progress_bar=True)
            batch = [
                {"id": key, "values": embedding.cpu().numpy().tolist(), "metadata": {"title": title}}
                for key, embedding, title in zip(keys, embeddings, titles)
            ]
            upsert_to_pinecone(pinecone_index, batch)
            print(f"Upserted final batch of size {len(images)}")

In [20]:
link_to_test_json = "output/test_images.json"

link= "output'/product_details_array.json"

In [ ]:
import ijson

filename = 'output/product_details_array.json'
with open(filename, 'rb') as f:
    objects = ijson.items(f, 'item')
    for obj in objects:
        print(obj['id'], obj['title'], obj['image_link'])

In [32]:
with open(link_to_test_json,"r") as f:
    items = ijson.items(f, '')

    for item in items:
        print(item)
        break

{'0000099813': {'image_link': 'https://m.media-amazon.com/images/I/3196g3bMM5L._AC_.jpg', 'title': 'Military US Issue JP 8 Gore TEX Fuel Handler Coverall L/R (Large)'}, '0310807115': {'image_link': 'https://m.media-amazon.com/images/I/41KXCJAX8HL._AC_.jpg', 'title': 'Messenger Bag with Detachable Cover Black LG'}, '0512238944': {'image_link': 'https://m.media-amazon.com/images/I/41DGFNVLwsL._AC_.jpg', 'title': 'Womens Cap Sleeve Crew Neck Tshirts with Pocket Summer Casual Leopard Loose Fit Side Split Tees Tops Pink'}, '0512238995': {'image_link': 'https://m.media-amazon.com/images/I/411qklz7joL._AC_.jpg', 'title': 'Womens Cap Sleeve Crew Neck Tshirts with Pocket Summer Casual Leopard Loose Fit Side Split Tees Tops Blue'}, '0512239045': {'image_link': 'https://m.media-amazon.com/images/I/41XR6Q-hwpL._AC_.jpg', 'title': 'Womens Cap Sleeve Crew Neck Tshirts with Pocket Summer Casual Leopard Loose Fit Side Split Tees Tops Khaki'}, '0512239134': {'image_link': 'https://m.media-amazon.com/im